# 05. Model Training Setup

Given a dataset of labelled training images, create a label map configuration file and Tensorflow train/test tfrecords to be used in the training of a variety of potential object detection models from the TensorFlow 2 Model Garden
 

## Configuration

Any configuration that is required to run this notebook can be customized in the next cell

In [1]:
# Prefix that will be included as a suffix in the label map file and tfrecord train and test files
# Please ensure that you have put the required train and test images, with their label files from
# lableImg, into the directory TensorFlow/workspace/imsages/train_XXX and test_XXX, where XXX is
# the dataset_version you specify below

dataset_version = 'V1'

## Code

In [2]:
# General imports
import os
import sys

from pathlib import Path

import wget

# Make sure local modules can be imported
module_path_root = os.path.abspath(os.pardir)
if module_path_root not in sys.path:
    sys.path.append(module_path_root)
    
# Get root install path, a level above the minor_thesis folder from GitHub
install_path_root = Path(module_path_root).parent.absolute()

In [3]:
# Derived paths
api_dir             = os.path.join(install_path_root, 'TensorFlow', 'models')
protoc_dir          = os.path.join(install_path_root, 'TensorFlow', 'protoc')
annotations_dir     = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'annotations')
image_train_dir     = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'images', 'train_{0:s}'.format(dataset_version))
image_test_dir      = os.path.join(install_path_root, 'TensorFlow', 'workspace', 'images', 'test_{0:s}'.format(dataset_version))

label_map_path      = os.path.join(annotations_dir, 'label_map_{0:s}.pbtxt'.format(dataset_version))
train_record_path   = os.path.join(annotations_dir, 'train_{0:s}.record'.format(dataset_version))
test_record_path    = os.path.join(annotations_dir, 'test_{0:s}.record'.format(dataset_version))

tf_record_script    = os.path.join(module_path_root, 'contrib', 'generate_tfrecord.py')
verification_script = os.path.join(api_dir, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')

# Create directories if they do not already exist
Path(annotations_dir).mkdir(parents=True, exist_ok=True)
Path(protoc_dir).mkdir(parents=True, exist_ok=True)
Path(image_train_dir).mkdir(parents=True, exist_ok=True)
Path(image_test_dir).mkdir(parents=True, exist_ok=True)

In [4]:
# Download the TensorFlow Model Garden API if it has not already been downloaded
# This step may take a while if it has not already been run, otherwise it will
# return quickly
if not os.path.exists(os.path.join(api_dir, 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {api_dir}

In [5]:
# Install Tensorflow Object Detection if it has not already been installed
# This step may also take a few moments if it has not been run before
# Ignore the message about "The system cannot find the path specified"
# as long as it eventually says "Successfully installed slim"
if not os.path.exists(os.path.join(protoc_dir, 'readme.txt')):
    if os.name=='posix':  
        !apt-get install protobuf-compiler
        !cd TensorFlow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
    if os.name=='nt':
        url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
        wget.download(url)
        !move protoc-3.15.6-win64.zip {protoc_dir}
        !cd {protoc_dir} && tar -xf protoc-3.15.6-win64.zip
        os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(protoc_dir, 'bin'))
        !cd TensorFlow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
        slim_path = os.path.join(install_path_root, 'TensorFlow', 'models', 'research', 'slim')
        !cd {slim_path} && pip install -e . 

In [6]:
# Run a verification script to check that TensorFlow is installed and working
# This script may return some errors due to issues with release management, but check for missing dependencies
# And check that it says the GPU is found, if there is one
!python {verification_script}

2021-10-08 21:19:52.991600: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
Running tests under Python 3.8.11: D:\ProgramData\Anaconda3\envs\gpuEnv\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-10-08 21:19:56.386109: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library nvcuda.dll
2021-10-08 21:19:56.421573: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 11.00GiB deviceMemoryBandwidth: 573.69GiB/s
2021-10-08 21:19:56.421591: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cudart64_101.dll
2021-10-08 21:19:56.424324: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library cublas64_1

[       OK ] ModelBuilderTF2Test.test_create_ssd_fpn_model_from_config
[ RUN      ] ModelBuilderTF2Test.test_create_ssd_models_from_config
I1008 21:20:01.073379 18824 ssd_efficientnet_bifpn_feature_extractor.py:142] EfficientDet EfficientNet backbone version: efficientnet-b0
I1008 21:20:01.073851 18824 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 64
I1008 21:20:01.073851 18824 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num iterations: 3
I1008 21:20:01.120306 18824 efficientnet_model.py:147] round_filter input=32 output=32
I1008 21:20:01.308533 18824 efficientnet_model.py:147] round_filter input=32 output=32
I1008 21:20:01.308533 18824 efficientnet_model.py:147] round_filter input=16 output=16
I1008 21:20:01.371525 18824 efficientnet_model.py:147] round_filter input=16 output=16
I1008 21:20:01.372018 18824 efficientnet_model.py:147] round_filter input=24 output=24
I1008 21:20:01.566127 18824 efficientnet_model.py:147] round_fi

In [7]:
# Create Label Map file
labels = [
    {'name':'BikeLaneMarker', 'id':1},
    {'name':'GiveWayMarker',  'id':2},
    {'name':'IslandMarker',   'id':3},
    {'name':'ArrowMarker',    'id':4},
    {'name':'RoadDefect',     'id':5},
    {'name':'RoadWriting',    'id':6}
]

with open(label_map_path, 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

I1008 21:20:21.555769 18824 test_util.py:1972] time(__main__.ModelBuilderTF2Test.test_unknown_meta_architecture): 0.0s
[       OK ] ModelBuilderTF2Test.test_unknown_meta_architecture
[ RUN      ] ModelBuilderTF2Test.test_unknown_ssd_feature_extractor
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_unknown_ssd_feature_extractor): 0.0s
I1008 21:20:21.556761 18824 test_util.py:1972] time(__main__.ModelBuilderTF2Test.test_unknown_ssd_feature_extractor): 0.0s
[       OK ] ModelBuilderTF2Test.test_unknown_ssd_feature_extractor
ERROR: test_create_center_net_model0 (customize_head_params=True) (__main__.ModelBuilderTF2Test)
ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
test_create_center_net_model(customize_head_params=True)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "D:\ProgramData\Anaconda3\envs\gpuEnv\lib\site-packages\absl\testing\parameterized.py", line 314, in bound_param_test
  

In [8]:
# Create TensorFlow records from the images in the train and test directories
# This step will take at least a few seconds, depending on the volume of images in the dataset
!python {tf_record_script} -x {image_train_dir} -l {label_map_path} -o {train_record_path}
!python {tf_record_script} -x {image_test_dir}  -l {label_map_path} -o {test_record_path}

Successfully created the TFRecord file: E:\Release\TensorFlow\workspace\annotations\train_V1.record
Successfully created the TFRecord file: E:\Release\TensorFlow\workspace\annotations\test_V1.record
